# Accounting for repeated purchase in Bass model

Suppose that already adopters will purchase $k$ units of products each period after adoption. ($k$ can non-integer). The total sales of each period is: 
$$S(t) = N(t) +k*A(t-1)$$
We typically have $S(t)$ in our data, but estimating Bass model requires us to know $N(t)$. <br>
**Question: how to get $N(t)$ from $S(t)$?**

We can use an inductive method:
- Create two arrays $A(t-1)$ and $N(t)$
- When $t=1$, $A(t-1)=A(0)=0$ and $N(t)=N(1)=S(1)$
- When $t>1$:
    - Calculate $A(t-1) = A(t-2) + N(t-1)$.
    - Calculate $N(t) = S(t)-k*A(t-1)$

Let us use the example of https://raw.githubusercontent.com/zoutianxin1992/MarketingAnalyticsPython/main/Marketing%20Analytics%20in%20Python/Bass%20model/Dataset/3-2-3%20repeated%20purcahse%201.csv. Suppose already adopters will buy 0.5 units of products every period after adoption (i.e. $k=0.5$).

In [55]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import least_squares               

# import historical data 
url = "https://raw.githubusercontent.com/zoutianxin1992/MarketingAnalyticsPython/main/Marketing%20Analytics%20in%20Python/Bass%20model/Dataset/3-2-3%20repeated%20purcahse%201.csv"
df = pd.read_csv(url) 
# Rename the variables to t and N_t
df.head()

,t,Sales
0,1,0.100
1,2,0.200
2,3,0.324
3,4,0.498
4,5,0.750


We will construct two two variables N(t) and A(t-1). Then fill in their values when $t=1$

In [56]:
T = len(df["Sales"]) 
k=0.5
df[["N","Atminus1"]] = 0
df["N"].iat[0] = df["Sales"].iat[0]
df.head()

,t,Sales,N,Atminus1
0,1,0.100,0.1,0
1,2,0.200,0.0,0
2,3,0.324,0.0,0
3,4,0.498,0.0,0
4,5,0.750,0.0,0


Next we calculate $N(t)$ and $A(t-1)$ inductively for $t=2,3,...,T$ . 

In [57]:
for i in range(1,T):
    df["Atminus1"].iat[i] = df["Atminus1"].iat[i-1] + df["N"].iat[i-1]
    df["N"].iat[i] = df["Sales"].iat[i] - k * df["Atminus1"].iat[i]
df.head()

,t,Sales,N,Atminus1
0,1,0.100,0.10000,0.0000
1,2,0.200,0.15000,0.1000
2,3,0.324,0.19900,0.2500
3,4,0.498,0.27350,0.4490
4,5,0.750,0.38875,0.7225


Finally we estimate $p,q,M$ using the $N(t)$ data we just obtained.

In [58]:
# define A_hat(t) and N_hat(t)

def A_hat(t,p,q,M):  #t: time, params: the 1*3 array for (p,q,M)
    return M * (1 - np.exp(-(p+q)*t))/(1 + q / p* np.exp(-(p+q)*t))

# define N_hat(t) 
def N_hat(t,p,q,M):  
    return A_hat(t,p,q,M) - A_hat(t-1,p,q,M)  # We can use the A_hat function to calculate N_hat instead of manually typing the formula

In [59]:
def prediction_error(params):   # Note that we input p,q,M as a 1*3 array "params." This is required by Python's NLS solver we will use. 
    p = params[0]
    q = params[1]
    M = params[2]
    Nhat = [N_hat(t,p,q,M) for t in range(1,T+1)]            # Given p,q,M, generate Bass prediction for each period
    return df["N"] - Nhat                                 # The output is an array of prediction error for each period

In [60]:
# estimate p,q,M using least_squares
# Bass model requires 0<p<1, 0<q<1, M>0, so we need to add the constraints
A_t = sum(df['N'])           # calculate already adopters until period t
params0 = [0.01,0.16,3*A_t]  # initial guess for p,q,M. Required by least_squares
estim_results= least_squares(prediction_error, params0, bounds = (0,np.Inf) )

#########################
# prediction_error: an array of prediction errors for each period
# param0: initial guesses
# bounds: The bounds for p,q,M. In our case p,q,M>0
#########################
# store the estimation results
p_estim = estim_results.x[0]
q_estim = estim_results.x[1]
M_estim = estim_results.x[2]
print(p_estim, q_estim, M_estim)
# Make sure "success" is True
# "x" is the estimated parameters (what we want).
# We don't worry about other parts for our purpose.

0.0008059713928382729 0.4203271972444095 69.26103440333753


Then we can predict the new adoption, the total adoption, and the sales in any period with the estimated $p,q, M$. For example, if we predict the sales in period 20, we can use $S(20)=N(20)+k*A(19)$.

In [63]:
A_hat(19,p_estim,q_estim,M_estim)* k + N_hat(20,p_estim,q_estim,M_estim)

32.65280315494652